# 🤖 **Módulo 5: Agents - Os Funcionários Inteligentes**

## **Aula 5.1: Agents Básicos - Delegando Tarefas**

---

### **Tá, mas o que são Agents?**

Imagina que você é um **chefe de uma empresa** e tem funcionários especializados:

**Sem Agents**: Você tem que fazer tudo sozinho - pesquisar, calcular, escrever, analisar...
**Com Agents**: Você delega para funcionários especializados que sabem usar ferramentas!

**Agent** = IA que pode **tomar decisões** e **usar ferramentas** para completar tarefas.

### **Por que Agents são Poderosos?**

**Sem Agents**: IA só conversa, não pode fazer ações
**Com Agents**: IA pode pesquisar, calcular, executar código, etc.

É como a diferença entre **conversar com alguém** e **ter um assistente que pode usar computador, calculadora, internet...**

---

<img src="https://www.shutterstock.com/image-photo/delegation-concept-figurine-boss-employees-600nw-2444400673.jpg" img>

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Coloque aqui os módulos e bibliotecas :)

In [ ]:
# Coloque aqui a célula universal de API :)

### **�� Setup Completo - Tudo Configurado para Funcionar**

**O que acabamos de configurar:**
- ✅ **Bibliotecas importadas**: Todas as ferramentas necessárias para agents
- ✅ **API configurada**: Modelo GPT-3.5-turbo funcionando
- ✅ **Ferramentas básicas**: Search tool pronta para uso
- ✅ **Memory configurado**: Para lembrar das conversas

**💡 Importante**: O `temperature=0.7` significa que o agente será criativo mas não muito aleatório. É como ter um funcionário inteligente que pensa antes de agir!

**�� Próximo passo**: Vamos criar nossa primeira ferramenta e ver o agente em ação!

### **Criando Nossa Primeira Ferramenta**

Vamos criar um **agente de pesquisa** que pode buscar informações na internet:

In [ ]:
# Criando ferramenta de pesquisa
# Como dar uma calculadora para o funcionário

from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Ferramenta de pesquisa
search_tool = DuckDuckGoSearchRun()

# Criar ferramentas com descrição
tools = [
    Tool(
        name="Pesquisa na Internet",
        func=search_tool.run,
        description="Use para buscar informações atualizadas na internet"
    )
]

# Memory para o agente
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Template para o agente
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente útil que pode pesquisar na internet."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# NOVO: Criar agente com sintaxe moderna
agent = create_openai_functions_agent(llm, tools, prompt)

# NOVO: Executor do agente
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

print("Agente de pesquisa criado!")
print("🌐 Pode buscar informações na internet")

### **🔍 Entendendo o Código - Peça por Peça**

**O que cada parte faz:**

1. **Tool Creation**:
   - `Tool(name="Pesquisa na Internet", ...)` - Cria uma ferramenta com nome e descrição
   - É como dar um nome e instruções para uma ferramenta

2. **Memory Setup**:
   - `ConversationBufferMemory` - Lembra de tudo que foi dito
   - Como um gravador que nunca para

3. **Prompt Template**:
   - Define como o agente deve se comportar
   - É como dar um manual de instruções para o funcionário

4. **Agent Creation**:
   - `create_openai_functions_agent` - Cria o agente com sintaxe moderna
   - Mais limpo que o método antigo

**💡 Dica**: A sintaxe `|` (pipe) é mais moderna, mas aqui usamos o método tradicional para clareza!

In [ ]:
# Testando o agente de pesquisa
# Vamos ver ele pesquisando!

print("TESTE: AGENTE DE PESQUISA")
print("=" * 50)

perguntas = [
    "Qual é a capital do Brasil?",
    "Quem é o presidente atual do Brasil?",
    "Qual a população de São Paulo?"
]

for i, pergunta in enumerate(perguntas, 1):
    print(f"\n❓ Pergunta {i}: {pergunta}")

    try:
        # NOVO: Usar agent_executor.invoke() em vez de agent.run()
        resposta = agent_executor.invoke({"input": pergunta})

        # NOVO: Acessar o conteúdo da resposta
        print(f"🤖 Resposta: {resposta['output'][:200]}...")

    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

### **�� Análise dos Resultados - O que Aconteceu?**

**O que o agente fez:**
- ✅ **Pesquisou na internet** usando DuckDuckGo
- ✅ **Processou as informações** encontradas
- ✅ **Respondeu de forma contextualizada** baseada nos dados reais
- ✅ **Manteve o histórico** das conversas

**Por que é impressionante:**
- 🧠 **Tomou decisões**: Escolheu usar a ferramenta de pesquisa
- 🔍 **Executou ações**: Realmente buscou na internet
- 💭 **Processou resultados**: Não só copiou, mas interpretou
- 📝 **Respondeu naturalmente**: Como um assistente humano

**💡 Diferença para ChatGPT**: ChatGPT só "sabe" o que foi treinado. Este agente pode buscar informações **ATUALIZADAS**!

## **Aula 5.2: Ferramentas Customizadas - Swiss Army Knife da IA**

### **Criando Ferramentas Personalizadas**

Vamos criar ferramentas customizadas. É como dar **ferramentas específicas** para cada funcionário:

In [ ]:
# Criando ferramentas customizadas
# Como dar ferramentas específicas para cada tarefa

from langchain.tools import BaseTool
from typing import Optional
import math

# Ferramenta de cálculo
class CalculadoraTool(BaseTool):
    name = "calculadora"
    description = "Use para fazer cálculos matemáticos"

    def _run(self, query: str) -> str:
        try:
            # Avalia expressão matemática com segurança
            allowed_names = {
                'abs': abs, 'round': round, 'min': min, 'max': max,
                'sum': sum, 'pow': pow, 'sqrt': math.sqrt
            }
            result = eval(query, {"__builtins__": {}}, allowed_names)
            return f"Resultado: {result}"
        except Exception as e:
            return f"Erro no cálculo: {e}"

# Ferramenta de análise de texto
class AnalisadorTextoTool(BaseTool):
    name = "analisador_texto"
    description = "Use para analisar textos (contar palavras, caracteres, etc.)"

    def _run(self, texto: str) -> str:
        palavras = len(texto.split())
        caracteres = len(texto)
        linhas = len(texto.split('\n'))

        return f"Análise do texto:\n- Palavras: {palavras}\n- Caracteres: {caracteres}\n- Linhas: {linhas}"

print("🔧 Ferramentas customizadas criadas!")
print("🧮 Calculadora e Analisador de Texto prontos")

### **🛠️ Ferramentas Customizadas - Por que são Poderosas?**

**O que criamos:**

1. **CalculadoraTool**:
   - ✅ Faz cálculos matemáticos complexos
   - ✅ Segura (não executa código malicioso)
   - ✅ Pode usar funções como sqrt, pow, etc.

2. **AnalisadorTextoTool**:
   - ✅ Conta palavras, caracteres, linhas
   - ✅ Análise rápida de qualquer texto
   - ✅ Útil para relatórios e documentação

**Por que customizar ferramentas:**
- �� **Específicas para sua necessidade**: Não depende de ferramentas genéricas
- ⚡ **Mais rápidas**: Otimizadas para tarefas específicas
- 🔒 **Mais seguras**: Você controla o que pode fazer
- �� **Mais precisas**: Resultados exatos para suas necessidades

**�� Analogia**: É como ter ferramentas específicas para cada trabalho em vez de uma chave de fenda universal!

In [ ]:
# Criando agente com ferramentas customizadas
# Como montar uma equipe completa

tools_avancadas = [
    Tool(
        name="Pesquisa na Internet",
        func=search_tool.run,
        description="Use para buscar informações atualizadas"
    ),
    Tool(
        name="Calculadora",
        func=CalculadoraTool()._run,
        description="Use para fazer cálculos matemáticos"
    ),
    Tool(
        name="Analisador de Texto",
        func=AnalisadorTextoTool()._run,
        description="Use para analisar textos"
    )
]

# NOVO: Criar agente com sintaxe moderna
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Template para o agente avançado
prompt_avancado = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente inteligente que pode pesquisar, calcular e analisar textos. Use as ferramentas disponíveis quando necessário."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# NOVO: Criar agente com sintaxe moderna
agent_avancado = create_openai_functions_agent(llm, tools_avancadas, prompt_avancado)

# NOVO: Executor do agente
agent_executor_avancado = AgentExecutor(
    agent=agent_avancado,
    tools=tools_avancadas,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=True
)

print("🤖 Agente avançado criado!")
print("🛠️  Pode pesquisar, calcular e analisar textos")

### **🤖 Agente Avançado - A Equipe Completa**

**O que temos agora:**
- 🔍 **Pesquisador**: Busca informações na internet
- 🧮 **Matemático**: Faz cálculos complexos
- �� **Analista**: Analisa textos e dados
- 🧠 **Coordenador**: Decide qual ferramenta usar

**Como funciona a tomada de decisão:**
1. **Recebe a pergunta** do usuário
2. **Analisa o que precisa** fazer
3. **Escolhe a ferramenta** mais adequada
4. **Executa a tarefa** usando a ferramenta
5. **Processa o resultado** e responde

**�� Exemplo**: Se você perguntar "Quantas pessoas vivem em São Paulo e qual a densidade populacional?", o agente vai:
- Usar a ferramenta de pesquisa para buscar a população
- Usar a calculadora para calcular a densidade
- Combinar tudo em uma resposta coerente!

In [ ]:
# Testando o agente avançado
# Vamos ver ele usando todas as ferramentas!

print("TESTE: AGENTE AVANÇADO")
print("=" * 50)

tarefas = [
    "Pesquise sobre a população do Brasil e calcule quantas pessoas isso representa por km²",
    "Analise este texto: 'Python é uma linguagem de programação incrível para iniciantes.'",
    "Calcule a raiz quadrada de 144 e depois pesquise sobre o número 12 na matemática"
]

for i, tarefa in enumerate(tarefas, 1):
    print(f"\n📋 Tarefa {i}: {tarefa}")

    try:
        # NOVO: Usar agent_executor_avancado.invoke() em vez de agent_avancado.run()
        resultado = agent_executor_avancado.invoke({"input": tarefa})

        # NOVO: Acessar o conteúdo da resposta
        print(f"Resultado: {resultado['output'][:300]}...")

    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

### **🎯 Análise das Tarefas Complexas - O Poder dos Agents**

**Tarefa 1**: "Pesquise sobre a população do Brasil e calcule quantas pessoas isso representa por km²"
- 🧠 **Decisão**: Precisa pesquisar E calcular
- 🔍 **Ação 1**: Usa ferramenta de pesquisa para população e área
- 🧮 **Ação 2**: Usa calculadora para divisão
- �� **Resultado**: Resposta completa e precisa

**Tarefa 2**: "Analise este texto: 'Python é uma linguagem de programação incrível para iniciantes.'"
- 🧠 **Decisão**: Análise de texto simples
- �� **Ação**: Usa analisador de texto
- �� **Resultado**: Estatísticas do texto

**Tarefa 3**: "Calcule a raiz quadrada de 144 e depois pesquise sobre o número 12 na matemática"
- 🧠 **Decisão**: Cálculo + pesquisa
- �� **Ação 1**: Calculadora para raiz quadrada
- 🔍 **Ação 2**: Pesquisa sobre o número 12
- 🔗 **Resultado**: Informações conectadas e contextualizadas

**💡 O que torna isso incrível**: O agente **entende** o que você quer e **coordena** múltiplas ferramentas automaticamente!

## **Aula 5.3: Agente Financeiro - Tomada de Decisão Inteligente**

### **Criando um Agente Especializado**

Vamos criar um **agente financeiro** que pode analisar investimentos e dar conselhos:

In [ ]:
# Ferramenta de análise financeira
class AnaliseFinanceiraTool(BaseTool):
    name = "analise_financeira"
    description = "Use para analisar investimentos e dar conselhos financeiros"

    def _run(self, pergunta: str) -> str:
        # Simulação de análise financeira
        if "cdb" in pergunta.lower():
            return "CDB é um investimento de renda fixa seguro. Rentabilidade: 100-110% do CDI. Ideal para conservadores."
        elif "ações" in pergunta.lower():
            return "Ações são investimentos de renda variável. Maior risco, maior potencial de retorno. Ideal para arrojados."
        elif "tesouro" in pergunta.lower():
            return "Tesouro Direto é o investimento mais seguro do Brasil. Garantido pelo governo federal."
        else:
            return "Para investimentos, considere seu perfil de risco e objetivos. Diversificação é fundamental."

# Ferramenta de cálculo de juros compostos
class JurosCompostosTool(BaseTool):
    name = "juros_compostos"
    description = "Use para calcular juros compostos e projeções financeiras"

    def _run(self, query: str) -> str:
        try:
            # Exemplo: calcular juros compostos
            if "calcular" in query.lower():
                # Simulação simples
                principal = 1000
                taxa = 0.01  # 1% ao mês
                tempo = 12   # 12 meses

                montante = principal * (1 + taxa) ** tempo
                return f"Investimento de R$ {principal} por {tempo} meses a {taxa*100}% ao mês = R$ {montante:.2f}"
            else:
                return "Use 'calcular' para fazer projeções financeiras"
        except:
            return "Erro no cálculo financeiro"

print("💰 Ferramentas financeiras criadas!")
print("📊 Análise financeira e cálculos de juros compostos")

### **💰 Ferramentas Financeiras - Especialização Inteligente**

**O que criamos:**

1. **AnaliseFinanceiraTool**:
   - ✅ Conhece diferentes tipos de investimentos
   - ✅ Dá conselhos baseados no perfil de risco
   - ✅ Explica conceitos financeiros de forma simples

2. **JurosCompostosTool**:
   - ✅ Calcula projeções financeiras
   - ✅ Mostra o poder dos juros compostos
   - ✅ Ajuda no planejamento financeiro

**Por que especialização é importante:**
- 🎯 **Conhecimento profundo**: Não é um "faz-tudo", é um especialista
- �� **Conselhos precisos**: Baseado em conhecimento financeiro real
- 📊 **Cálculos corretos**: Matemática financeira precisa
- 🚀 **Eficiência**: Não perde tempo com ferramentas inadequadas

**�� Analogia**: É como ter um consultor financeiro especializado em vez de um assistente geral que "sabe um pouco de tudo"!

In [ ]:
# Criando agente financeiro
# Como contratar um consultor financeiro inteligente

tools_financeiras = [
    Tool(
        name="Pesquisa Financeira",
        func=search_tool.run,
        description="Use para buscar informações sobre investimentos e mercado financeiro"
    ),
    Tool(
        name="Análise Financeira",
        func=AnaliseFinanceiraTool()._run,
        description="Use para analisar investimentos e dar conselhos"
    ),
    Tool(
        name="Cálculo Financeiro",
        func=JurosCompostosTool()._run,
        description="Use para calcular juros compostos e projeções"
    )
]

# Template para o agente financeiro
prompt_financeiro = ChatPromptTemplate.from_messages([
    ("system", "Você é um consultor financeiro experiente e inteligente. Use as ferramentas disponíveis para dar conselhos precisos sobre investimentos, mercado financeiro e cálculos financeiros."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# NOVO: Criar agente com sintaxe moderna
agent_financeiro = create_openai_functions_agent(llm, tools_financeiras, prompt_financeiro)

# NOVO: Executor do agente
agent_executor_financeiro = AgentExecutor(
    agent=agent_financeiro,
    tools=tools_financeiras,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=True
)

print("💼 Agente financeiro criado!")
print("🎯 Consultor financeiro inteligente pronto")

### **💼 Agente Financeiro - O Consultor Inteligente**

**Características do nosso consultor:**
- 🧠 **Inteligente**: Usa GPT-3.5-turbo para entender contextos complexos
- �� **Bem informado**: Pode buscar informações atualizadas do mercado
- �� **Especializado**: Conhece profundamente investimentos e finanças
- 🧮 **Preciso**: Faz cálculos financeiros corretos
- 📝 **Didático**: Explica conceitos complexos de forma simples

**Como ele funciona:**
1. **Recebe sua pergunta** financeira
2. **Analisa o que você precisa** saber
3. **Escolhe as ferramentas** mais adequadas
4. **Executa pesquisas** se necessário
5. **Faz cálculos** se precisar
6. **Combina tudo** em um conselho coerente

**💡 Vantagem sobre consultores humanos**:
- ✅ Sempre disponível
- ✅ Sem custo por consulta
- ✅ Acesso a informações em tempo real
- ✅ Sem viés emocional

In [ ]:
# Testando o agente financeiro
# Vamos ver o consultor em ação!

print("💼 TESTE: AGENTE FINANCEIRO")
print("=" * 50)

consultas_financeiras = [
    "O que é CDB e é um bom investimento?",
    "Calcule quanto eu teria se investisse R$ 1000 por 12 meses",
    "Pesquise sobre as melhores ações para investir em 2024"
]

for i, consulta in enumerate(consultas_financeiras, 1):
    print(f"\n💰 Consulta {i}: {consulta}")

    try:
        # NOVO: Usar agent_executor_financeiro.invoke() em vez de agent_financeiro.run()
        resposta = agent_executor_financeiro.invoke({"input": consulta})

        # NOVO: Acessar o conteúdo da resposta
        print(f"💼 Consultor: {resposta['output'][:300]}...")

    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

### **📊 Análise das Consultas Financeiras - Consultoria Inteligente**

**Consulta 1**: "O que é CDB e é um bom investimento?"
- 🧠 **Análise**: Pergunta sobre conceito + avaliação
- �� **Ferramenta**: Análise financeira especializada
- 📊 **Resultado**: Explicação clara + recomendação personalizada

**Consulta 2**: "Calcule quanto eu teria se investisse R$ 1000 por 12 meses"
- 🧠 **Análise**: Cálculo financeiro específico
- 🧮 **Ferramenta**: Cálculo de juros compostos
- �� **Resultado**: Projeção financeira precisa

**Consulta 3**: "Pesquise sobre as melhores ações para investir em 2024"
- 🧠 **Análise**: Informação atualizada + recomendação
- 🔍 **Ferramenta**: Pesquisa na internet + análise financeira
- 🎯 **Resultado**: Informações atuais + contexto histórico

**💡 O que torna isso valioso**:
- �� **Automação**: Não precisa consultar múltiplas fontes
- ⚡ **Velocidade**: Respostas em segundos
- �� **Precisão**: Combina dados atuais com análise especializada
- �� **Educação**: Aprende enquanto recebe conselhos

### **Na Prática, Meu Consagrado!** 💪

**O que aprendemos sobre Agents:**

1. ✅ **Agents básicos** - IA que pode usar ferramentas
2. ✅ **Ferramentas customizadas** - Swiss Army Knife da IA
3. ✅ **Tomada de decisão** - Agents que escolhem qual ferramenta usar
4. ✅ **Especialização** - Agents para áreas específicas

### **Comparação: Com vs Sem LangChain**

**Sem LangChain**: IA só conversa, não pode fazer ações
**Com LangChain**: IA pode pesquisar, calcular, analisar, tomar decisões

<img src = 'https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fagent-framework.ad7f5098.png&w=1920&q=75' img>

**🎯 Próximo módulo**: Vamos aprender sobre **Document Loaders** - como fazer a IA ler documentos!

**💡 Resumo do Módulo 5**:
- ✅ Agents básicos e avançados
- ✅ Ferramentas customizadas
- ✅ Tomada de decisão inteligente
- ✅ Especialização por área

**🤖 Agora você tem funcionários inteligentes que podem usar ferramentas!**